In [ ]:
!pip install pandas

In [ ]:
!pip install kagglehub

In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
import sklearn 
import matplotlib.pyplot as plt
import kagglehub
import pandas as pd
import tensorflow as tf

In [ ]:
def age_to_agegroup(age):
    if age > 100:
        return 0;
    elif age > 90:
        return 1;
    elif age > 80:
        return 2;
    elif age > 70:
        return 3;    
    elif age > 60:
        return 4;
    elif age > 50:
        return 5;
    elif age > 40:
        return 6; 
    else:
        return 7;

def read_and_normalize_data():
    # Download most recent version of the individual-age-of-death-and-related-factors dataset from kaggle
    path = kagglehub.dataset_download("joannpineda/individual-age-of-death-and-related-factors")
    file = path + "/data.json";

    print("Data set is downloaded from kaggle-hub as:", file) 

    # Read data into panda
    kg = pd.read_json(file);

    # Convert pounds to kg and inches to centimeters
    print("Converting pounds to kg and inches to centimeters..") 
    kg.weight = kg.weight * 0.45359237;
    kg.height = kg.height * 2.54;
    
    # String values like 'f'/'m' (for female/male) and 'y'/'n' (for yes/no) to int8
    print("Converting chars to int..");
    kg.sex = kg.sex.astype('category').cat.codes;
    kg.smoker = kg.smoker.astype('category').cat.codes;
    kg.nic_other = kg.nic_other.astype('category').cat.codes;
    kg.occup_danger = kg.occup_danger.astype('category').cat.codes;
    kg.ls_danger = kg.ls_danger.astype('category').cat.codes;
    kg.cannabis = kg.cannabis.astype('category').cat.codes;
    kg.opioids = kg.opioids.astype('category').cat.codes;
    kg.other_drugs = kg.other_drugs.astype('category').cat.codes;
    kg.addiction = kg.addiction.astype('category').cat.codes;
    kg.diabetes = kg.diabetes.astype('category').cat.codes;
    kg.hds = kg.hds.astype('category').cat.codes;
    kg.asthma = kg.asthma.astype('category').cat.codes;
    kg.immune_defic = kg.immune_defic.astype('category').cat.codes;
    kg.family_cancer = kg.family_cancer.astype('category').cat.codes;
    kg.family_heart_disease = kg.family_heart_disease.astype('category').cat.codes;
    kg.family_cholesterol = kg.family_cholesterol.astype('category').cat.codes;

    # Round age to multiple of 5
    print("Rounding age up to 5 years");
    kg["age"] = kg.age.map(age_to_agegroup);

    return kg


In [ ]:
df = read_and_normalize_data();
df

In [ ]:
# shuffle
df = df.sample(frac=1)

# we can split the data the simple way because we know the set is shuffled
train = df[:8000];
test  = df[8000:];

# use all attributes except age (age has index 0)
x_train = train.iloc[:, 1:23]; 
x_test = test.iloc[:, 1:23];

# use the age as labels
y_train = train.age;
y_test = test.age;

print("x_train", x_train.shape);
print("y_train", y_test.shape);
print("x_test", x_train.shape);
print("y_test", y_test.shape);


In [ ]:
# The age's are still integers (0-17). We need these as vectors instead, so we can match them to the 
# 17 outputs of the neural network. 

# CHANGE * for information - the 17 labels and their meaning
#  0:  <30
#  1:   35
#  2:   40
#  3:   45
#  4:   50
#  5:   55
#  6:   60
#  7:   65
#  8:   70
#  9:   75
# 10:   80
# 11:   85
# 12:   90
# 13:   95
# 14:  100
# 15:  105
# 16: >110

from tensorflow.keras.utils import to_categorical

# change to 
print(y_train.shape, y_test.shape)
print(y_train)

y_train = to_categorical(y_train)
y_test  = to_categorical(y_test)

print(y_train.shape, y_test.shape)
print(y_train)

print("the 1 in the vector is the label for the age group")
y_train[0]

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.utils import plot_model
  
model = Sequential()

model.add(Dense(22))                     # first dense layer - 22 matches the number of features of the input
model.add(Activation('relu'))            # activation layer
model.add(Dense(50))                     # first dense layer - 22 matches the number of features of the input
model.add(Activation('relu'))            # activation layer
model.add(Dense(50))                     # first dense layer - 22 matches the number of features of the input
model.add(Activation('relu'))            # activation layer
model.add(Dense(8))                      # second dense layer - 8 matches the number of age labels

model.add(Activation('softmax'))         # output class probabilities for each node totaling upto 100%

model.summary()

In [ ]:
from tensorflow.keras.optimizers import SGD, Adam

optimizer = Adam(learning_rate=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train, epochs=15, batch_size=32, validation_split=1/6)

In [ ]:
def print_prediction(prediction):
    print("P R E D I C T I O N:")
    age = 30
    cummulative = 0.0;
    for p in prediction:
        cummulative = cummulative + p; 
        print("age", age, ":", format(cummulative,".0%"));
        age = age + 5;
        if cummulative > 0.95:
            break;

def print_input(input):
    print("I N P U T:")
    print(x_test.iloc[person_index])  

predictions = model.predict(x_test)

print(x_test.shape)
print(predictions.shape)

    


In [ ]:
def print_ACC(prediction):
    print("P R E D I C T I O N:")
    age = 30
    cummulative = 0.0;
    for p in prediction:
        cummulative = cummulative + p; 
        print("age", age, ":", format(cummulative,".0%"));
        age = age + 10;
        # if cummulative > 0.95:
        #     break;

def print_prediction(prediction):
    print("P R E D I C T I O N:")
    age = 30
    cummulative = 0.0;
    for p in prediction:
        print("age", age, ":", format(p,".0%"));
        age = age + 10;

def print_input(input):
    print("I N P U T:")
    print(x_test.iloc[person_index])  

predictions = model.predict(x_test)

print(x_test.shape)
print(predictions.shape)

In [ ]:
import seaborn as sns

def get_age(matrix):
    age_with_highest_perc = [];
    i = 0;
    for row in matrix:
        age = row.argmax(axis=0);
        # print(i, age);
        age_with_highest_perc.append(age);
        # print(i, age_with_highest_perc[i])
        i = i + 1;
    return age_with_highest_perc
    
actual_age = get_age(y_test);
predicted_age = get_age(predictions);

print(actual_age[:10]);
print(predicted_age[:10]);

sns.displot(x=actual_age, bins=[0, 1, 2, 3, 4, 5, 6, 7]); 
plt.show();

sns.displot(x=predicted_age, bins=[0, 1, 2, 3, 4, 5, 6, 7]);
plt.show();

plt.scatter(actual_age, predicted_age, s=400, alpha=0.02, color='red');
plt.show();

In [ ]:
person_index = 4;

print()
print_input(x_test.iloc[person_index])
print(y_test[person_index])
print()
print_prediction(predictions[person_index]);
print_ACC(predictions[person_index]);